# Lab 3: Contextual Bandit-Based News Article Recommendation

**`Course`:** Reinforcement Learning Fundamentals  
**`Student Name`:**  
**`Roll Number`:**  
**`GitHub Branch`:** firstname_U20230xxx  

# Imports and Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Import advanced classifiers
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

from rlcmab_sampler import sampler


# Load Datasets

In [ ]:
# Load datasets
news_df = pd.read_csv("data/news_articles.csv")
train_users = pd.read_csv("data/train_users.csv")
test_users = pd.read_csv("data/test_users.csv")

print("News Articles Dataset:")
print(news_df.head())
print(f"\nShape: {news_df.shape}")

print("\n" + "="*60)
print("Train Users Dataset:")
print(train_users.head())
print(f"\nShape: {train_users.shape}")


## Data Preprocessing

In this section:
- Handle missing values
- Encode categorical features
- Prepare data for user classification

In [ ]:
# Check for missing values in all datasets
print("=" * 60)
print("Missing Values Analysis")
print("=" * 60)
print("\nNews Articles Dataset:")
print(news_df.isnull().sum())
print(f"\nShape: {news_df.shape}")

print("\nTrain Users Dataset:")
print(train_users.isnull().sum())
print(f"\nShape: {train_users.shape}")

print("\nTest Users Dataset:")
print(test_users.isnull().sum())
print(f"\nShape: {test_users.shape}")

In [ ]:
# Handle missing values
# For numerical columns: fill with median
# For categorical columns: fill with mode

def handle_missing_values(df):
    """
    Handle missing values in the dataset.
    - Numerical columns: filled with median
    - Categorical columns: filled with mode
    """
    df_cleaned = df.copy()
    
    for col in df_cleaned.columns:
        if df_cleaned[col].isnull().sum() > 0:
            if df_cleaned[col].dtype in ['int64', 'float64']:
                # Fill numerical columns with median
                df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)
            else:
                # Fill categorical columns with mode
                df_cleaned[col].fillna(df_cleaned[col].mode()[0], inplace=True)
    
    return df_cleaned

# Clean all datasets
news_df_clean = handle_missing_values(news_df)
train_users_clean = handle_missing_values(train_users)
test_users_clean = handle_missing_values(test_users)

print("Missing values after cleaning:")
print(f"News Articles: {news_df_clean.isnull().sum().sum()}")
print(f"Train Users: {train_users_clean.isnull().sum().sum()}")
print(f"Test Users: {test_users_clean.isnull().sum().sum()}")

In [ ]:
# Display dataset information
print("=" * 60)
print("Dataset Information")
print("=" * 60)

print("\nNews Articles Categories:")
print(news_df_clean['category'].value_counts())

print("\nUser Categories (Train):")
print(train_users_clean['category'].value_counts())

print("\nColumn names (Train Users):")
print(train_users_clean.columns.tolist())

In [ ]:
# Feature encoding for user data
# Separate features and labels

# For training users
X_train_full = train_users_clean.drop('category', axis=1)
y_train_full = train_users_clean['category']

# For test users
X_test = test_users_clean.drop('category', axis=1) if 'category' in test_users_clean.columns else test_users_clean

# Identify categorical columns (non-numeric)
categorical_cols = X_train_full.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_train_full.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"Categorical columns: {categorical_cols}")
print(f"Numerical columns: {numerical_cols}")

In [ ]:
# Encode categorical features using LabelEncoder
label_encoders = {}

X_train_encoded = X_train_full.copy()
X_test_encoded = X_test.copy()

for col in categorical_cols:
    le = LabelEncoder()
    
    # Fit on training data
    X_train_encoded[col] = le.fit_transform(X_train_full[col].astype(str))
    
    # Transform test data (handle unseen labels)
    if col in X_test_encoded.columns:
        # Create a mapping for unseen categories
        test_col_values = X_test[col].astype(str)
        X_test_encoded[col] = test_col_values.map(
            lambda x: le.transform([x])[0] if x in le.classes_ else -1
        )
    
    label_encoders[col] = le

print("\nEncoding completed!")
print(f"Encoded training data shape: {X_train_encoded.shape}")
print(f"Encoded test data shape: {X_test_encoded.shape}")

In [ ]:
# Display sample of encoded data
print("Sample of encoded training data:")
print(X_train_encoded.head())
print("\nData types:")
print(X_train_encoded.dtypes)

In [ ]:
# Map news categories to standard format (Entertainment, Education, Tech, Crime)
# Based on the assignment requirements
print("=" * 60)
print("News Category Mapping")
print("=" * 60)

# The actual categories in the dataset might differ from the required ones
# We'll map them appropriately
category_mapping = {
    'U.S. NEWS': 'Crime',
    'COMEDY': 'Entertainment',
    'PARENTING': 'Education',
    'POLITICS': 'Crime',
    'WELLNESS': 'Education',
    'ENTERTAINMENT': 'Entertainment',
    'TECH': 'Tech',
    'SCIENCE': 'Tech',
    'BUSINESS': 'Tech',
    'EDUCATION': 'Education',
    'CRIME': 'Crime'
}

# Apply mapping if categories exist in news dataset
if 'category' in news_df_clean.columns:
    # Check unique categories
    print("\nOriginal categories in news dataset:")
    print(news_df_clean['category'].unique())
    
    # Map to standard categories
    news_df_clean['category_mapped'] = news_df_clean['category'].map(
        lambda x: category_mapping.get(x.upper() if isinstance(x, str) else x, x)
    )
    
    print("\nMapped categories:")
    print(news_df_clean['category_mapped'].value_counts())
    
    # Use mapped categories
    news_df_clean['category'] = news_df_clean['category_mapped']

# Define the arm mapping according to assignment specifications
print("\n" + "="*60)
print("Arm Index Mapping (j values)")
print("="*60)
arm_mapping = {
    0: ('Entertainment', 'user_1'),
    1: ('Education', 'user_1'),
    2: ('Tech', 'user_1'),
    3: ('Crime', 'user_1'),
    4: ('Entertainment', 'user_2'),
    5: ('Education', 'user_2'),
    6: ('Tech', 'user_2'),
    7: ('Crime', 'user_2'),
    8: ('Entertainment', 'user_3'),
    9: ('Education', 'user_3'),
    10: ('Tech', 'user_3'),
    11: ('Crime', 'user_3')
}

for arm_idx, (category, user) in arm_mapping.items():
    print(f"Arm {arm_idx:2d}: ({category:13s}, {user})")

# Create reverse mapping for easy lookup
category_user_to_arm = {v: k for k, v in arm_mapping.items()}
print("\nArm mapping created successfully!")

## User Classification

Train a classifier to predict the user category (`User1`, `User2`, `User3`),
which serves as the **context** for the contextual bandit.


In [ ]:
# Split the training data into train and validation sets (80-20 split)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_encoded, 
    y_train_full, 
    test_size=0.2, 
    random_state=42,
    stratify=y_train_full
)

print("Data Split Summary:")
print("=" * 60)
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Validation set size: {X_val.shape[0]} samples")
print(f"Test set size: {X_test_encoded.shape[0]} samples")
print("\nClass distribution in training set:")
print(y_train.value_counts())
print("\nClass distribution in validation set:")
print(y_val.value_counts())

In [ ]:
# Note: We will use state-of-the-art gradient boosting classifiers:
# - LightGBM: Fast, efficient, and excellent for structured data
# - XGBoost: Robust and widely used in competitions
# - CatBoost: Handles categorical features natively
print("Using advanced gradient boosting classifiers...")

In [ ]:
# LightGBM Classifier
print("=" * 60)
print("Training LightGBM Classifier")
print("=" * 60)

lgb_classifier = lgb.LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=7,
    num_leaves=31,
    random_state=42,
    verbose=-1
)

lgb_classifier.fit(X_train, y_train)

# Predictions
y_pred_lgb = lgb_classifier.predict(X_val)

# Evaluation
lgb_accuracy = accuracy_score(y_val, y_pred_lgb)
print(f"\nValidation Accuracy: {lgb_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred_lgb))
print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_pred_lgb))

In [ ]:
# XGBoost Classifier
print("=" * 60)
print("Training XGBoost Classifier")
print("=" * 60)

# Encode labels for XGBoost (requires 0, 1, 2 format)
label_encoder_y = LabelEncoder()
y_train_encoded = label_encoder_y.fit_transform(y_train)
y_val_encoded = label_encoder_y.transform(y_val)

xgb_classifier = xgb.XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=7,
    random_state=42,
    eval_metric='mlogloss',
    verbosity=0
)

xgb_classifier.fit(X_train, y_train_encoded)

# Predictions
y_pred_xgb_encoded = xgb_classifier.predict(X_val)
y_pred_xgb = label_encoder_y.inverse_transform(y_pred_xgb_encoded)

# Evaluation
xgb_accuracy = accuracy_score(y_val, y_pred_xgb)
print(f"\nValidation Accuracy: {xgb_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred_xgb))
print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_pred_xgb))

In [ ]:
# CatBoost Classifier
print("=" * 60)
print("Training CatBoost Classifier")
print("=" * 60)

catboost_classifier = CatBoostClassifier(
    iterations=200,
    learning_rate=0.05,
    depth=7,
    random_state=42,
    verbose=0
)

catboost_classifier.fit(X_train, y_train)

# Predictions
y_pred_cat = catboost_classifier.predict(X_val)

# Evaluation
cat_accuracy = accuracy_score(y_val, y_pred_cat)
print(f"\nValidation Accuracy: {cat_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred_cat))
print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_pred_cat))

In [ ]:
# Compare all classifiers
print("=" * 60)
print("Classifier Performance Comparison")
print("=" * 60)
print(f"LightGBM Accuracy:  {lgb_accuracy:.4f}")
print(f"XGBoost Accuracy:   {xgb_accuracy:.4f}")
print(f"CatBoost Accuracy:  {cat_accuracy:.4f}")

# Select the best classifier
classifiers = {
    'LightGBM': (lgb_classifier, lgb_accuracy),
    'XGBoost': (xgb_classifier, xgb_accuracy),
    'CatBoost': (catboost_classifier, cat_accuracy)
}

best_classifier_name = max(classifiers, key=lambda k: classifiers[k][1])
best_classifier = classifiers[best_classifier_name][0]

print(f"\n{'='*60}")
print(f"🏆 Best Classifier: {best_classifier_name}")
print(f"🎯 Best Accuracy: {classifiers[best_classifier_name][1]:.4f}")
print(f"{'='*60}")

In [ ]:
# Retrain the best classifier on the full training data for final use
print(f"\nRetraining {best_classifier_name} on full training data...")

if best_classifier_name == 'LightGBM':
    final_classifier = lgb.LGBMClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=7,
        num_leaves=31,
        random_state=42,
        verbose=-1
    )
    final_classifier.fit(X_train_encoded, y_train_full)
    
elif best_classifier_name == 'XGBoost':
    final_classifier = xgb.XGBClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=7,
        random_state=42,
        eval_metric='mlogloss',
        verbosity=0
    )
    y_train_full_encoded = label_encoder_y.fit_transform(y_train_full)
    final_classifier.fit(X_train_encoded, y_train_full_encoded)
    
elif best_classifier_name == 'CatBoost':
    final_classifier = CatBoostClassifier(
        iterations=200,
        learning_rate=0.05,
        depth=7,
        random_state=42,
        verbose=0
    )
    final_classifier.fit(X_train_encoded, y_train_full)

print(f"✅ {best_classifier_name} trained on full training data!")
print("\nThis classifier will be used as the Context Detector for the bandit system.")

In [ ]:
# Create a function to predict user context
def predict_user_context(user_features):
    """
    Predict the user category (user_1, user_2, or user_3) given user features.
    
    Args:
        user_features: DataFrame or array of user features
    
    Returns:
        Predicted user category
    """
    if best_classifier_name == 'XGBoost':
        # XGBoost returns encoded labels, need to inverse transform
        predictions_encoded = final_classifier.predict(user_features)
        return label_encoder_y.inverse_transform(predictions_encoded)
    else:
        return final_classifier.predict(user_features)

# Test the function on a sample
sample_prediction = predict_user_context(X_test_encoded.iloc[:5])
print("Sample predictions on test users:")
print(sample_prediction)

# Display the mapping for reference
print("\n" + "="*60)
print("User Context to Index Mapping (for Bandit):")
print("="*60)
user_to_index = {'user_1': 0, 'user_2': 1, 'user_3': 2}
for user, idx in user_to_index.items():
    print(f"{user} -> Index {idx}")

# `Contextual Bandit`

## Reward Sampler Initialization

The sampler is initialized using the student's roll number `i`.
Rewards are obtained using `sampler.sample(j)`.


## Arm Mapping

| Arm Index (j) | News Category | User Context |
|--------------|---------------|--------------|
| 0–3          | Entertainment, Education, Tech, Crime | User1 |
| 4–7          | Entertainment, Education, Tech, Crime | User2 |
| 8–11         | Entertainment, Education, Tech, Crime | User3 |

## Epsilon-Greedy Strategy

This section implements the epsilon-greedy contextual bandit algorithm.


## Upper Confidence Bound (UCB)

This section implements the UCB strategy for contextual bandits.

## SoftMax Strategy

This section implements the SoftMax strategy with temperature $ \tau = 1$.


## Reinforcement Learning Simulation

We simulate the bandit algorithms for $T = 10,000$ steps and record rewards.

P.S.: Change $T$ value as and if required.


## Results and Analysis

This section presents:
- Average Reward vs Time
- Hyperparameter comparisons
- Observations and discussion


## Final Observations

- Comparison of Epsilon-Greedy, UCB, and SoftMax
- Effect of hyperparameters
- Strengths and limitations of each approach
